# IMDB Project 3
- Juliana Sahagun
- 08/17/22

In [4]:
# Import libraries
import pandas as pd
import numpy as np
import json,os,time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
# Imports for MySQL Requirements
import pymysql
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists


In [2]:
# Download files
basics_url= "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url= "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url= "https://datasets.imdbws.com/title.akas.tsv.gz"

In [3]:
# Loading the data
df_basics = pd.read_csv(basics_url,sep='\t', low_memory=False)
df_akas= pd.read_csv(akas_url, sep='\t', low_memory=False)
df_ratings= pd.read_csv(ratings_url,sep='\t', low_memory=False)

# Part 1

In [4]:
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
df_ratings.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
df_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


## Cleaning/Filtering

Basics Preprocessing

In [7]:
# Replace "\N" with np.nan
df_basics = df_basics.replace({'\\N':np.nan})

In [8]:
# Remove movies having missing values for runtimeMinutes and genre
df_basics= df_basics.dropna(subset=['runtimeMinutes','genres','startYear'])

startYear also has null values that should be removed because it interferes with keep all movies with the start year 2000-2022

In [9]:
# Include only full-length movies (titleType = "movie")
df_basics = df_basics.loc[df_basics['titleType']=='movie']

In [10]:
# Eliminate movies that include "Documentary" in genre
doc= df_basics['genres'].str.contains('documentary', case=False)
df_basics = df_basics[~doc]

In [11]:
# Keep startYear 2000-2022
df_basics= df_basics.loc[(df_basics['startYear'] >=2000) & (df_basics['startYear'] <=2021)]

TypeError: '>=' not supported between instances of 'str' and 'int'

In [12]:
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 275324 entries, 8 to 9187478
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          275324 non-null  object
 1   titleType       275324 non-null  object
 2   primaryTitle    275324 non-null  object
 3   originalTitle   275324 non-null  object
 4   isAdult         275324 non-null  object
 5   startYear       275324 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  275324 non-null  object
 8   genres          275324 non-null  object
dtypes: object(9)
memory usage: 21.0+ MB


Akas Preprocessing

In [13]:
#Keep only movies that were released in the United States
df_akas = df_akas.loc[df_akas['region']=='US']

In [14]:
# Replace "\N" with np.nan
df_akas=df_akas.replace({'\\N': np.nan})

In [15]:
df_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1344834 entries, 5 to 33021300
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1344834 non-null  object
 1   ordering         1344834 non-null  int64 
 2   title            1344834 non-null  object
 3   region           1344834 non-null  object
 4   language         3690 non-null     object
 5   types            963701 non-null   object
 6   attributes       44806 non-null    object
 7   isOriginalTitle  1343459 non-null  object
dtypes: int64(1), object(7)
memory usage: 92.3+ MB


Ratings Preprocessing

In [16]:
# Replace "\N" with np.nan
df_ratings = df_ratings.replace({'\\N':np.nan})

In [17]:
#Filtering one dataframe based on another
keepers = df_basics['tconst'].isin(df_akas['titleId'])
df_basics =df_basics[keepers]
df_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
1273,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"
...,...,...,...,...,...,...,...,...,...
9186765,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama
9187161,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
9187301,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9187310,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"


In [18]:
# Make folders for data
import os
os.makedirs('Data/',exist_ok=True) 

# Confirm folder created
os.listdir("Data/")


['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [19]:
# Save current dataframe to file
df_basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
df_akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
df_ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [22]:
# Open saved file and preview again
df_basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
df_basics.head()



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
3,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
4,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"


In [21]:
df_ratings=pd.read_csv("Data/title_ratings.csv.gz", low_memory =False)
df_ratings.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short"


In [23]:
df_akas=pd.read_csv("Data/title_akas.csv.gz",low_memory=False)
df_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


# Part 2

In [26]:
# Open and read file
with open('/Users/julia/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [27]:
tmdb.API_KEY =  login['api-key']

Function below adds the certification (MPGG Rating) to movie.info

In [28]:
# Code adapted from the LP
def get_movie_and_rating(movie_id):    
    
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [29]:
# Test function on these two movies to make sure it runs correctly
test = get_movie_and_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 262.534,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [30]:
test2 = get_movie_and_rating("tt0332280")
test2

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 73.519,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [31]:
# Create a folder in data for each year
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [32]:
YEARS_TO_GET =[2000,2001]

In [33]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [39]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [40]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
            
            #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc=f'Movies from {YEAR}',
                                      position=1,
                                      leave=True):
            # Attempt to retrieve then data for the movie id
            try:
                temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
                # Append/extend results to existing file using a pre-made function
                write_json(temp,JSON_FILE)
                # Short 20 ms sleep to prevent overwhelming server
                time.sleep(0.02)

            # If it fails,  make a dict with just the id and None for certification.
            except Exception as e:
                continue

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1410 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1529 [00:00<?, ?it/s]

# Part 3

## **Normalizing Genres**

In [8]:
# Load data again
# Load title basics file 
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

# Load title rating file 
rating_df = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

# Load in your csv.gz's of results for combined years
combined = pd.read_csv("Data/tmdb_results_combined.csv.gz", low_memory = False)

ParserError: Error tokenizing data. C error: out of memory

In [5]:
# Convert the single string of genres from title basics into 2 new tables

# create a col with a list of genres
basics_df['genres_split'] = basics_df['genres'].str.split(',')
basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance,[Romance]
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography","[Action, Adventure, Biography]"
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama,[Drama]
3,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy","[Adventure, Fantasy]"
4,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family","[Biography, Drama, Family]"
...,...,...,...,...,...,...,...,...,...,...
151098,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama,[Drama]
151099,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
151100,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama,[Drama]
151101,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [6]:
# exploding the column of lists
exploded = basics_df.explode('genres_split')
exploded

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,NaN,45,Romance,Romance
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography",Action
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography",Adventure
1,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography",Biography
2,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...
151101,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Action
151101,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Adventure
151101,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Thriller
151102,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History",Drama


In [7]:
# saving the unique values from the exploded column
unique_genres = sorted(exploded['genres_split'].dropna().unique())
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [8]:
# Create a new title_genres table
title_genres = exploded[['tconst','genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0000009,Romance
1,tt0000574,Action
1,tt0000574,Adventure
1,tt0000574,Biography
2,tt0000591,Drama


In [9]:
#  Create a genre mapper dictionary to replace string genres with integers
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Film-Noir': 10,
 'Game-Show': 11,
 'History': 12,
 'Horror': 13,
 'Music': 14,
 'Musical': 15,
 'Mystery': 16,
 'News': 17,
 'Reality-TV': 18,
 'Romance': 19,
 'Sci-Fi': 20,
 'Short': 21,
 'Sport': 22,
 'Talk-Show': 23,
 'Thriller': 24,
 'War': 25,
 'Western': 26}

In [10]:
# Create a new genre_id column using the genre_map dictionary with either .map or .replace.
title_genres['genre_id'] = title_genres['genres_split'].map(genre_map)

title_genres = title_genres.drop(columns='genres_split')
title_genres

,tconst,genre_id
0,tt0000009,19
1,tt0000574,0
1,tt0000574,2
1,tt0000574,4
2,tt0000591,7
...,...,...
151101,tt9916190,0
151101,tt9916190,2
151101,tt9916190,24
151102,tt9916362,7


In [11]:
# Convert the genre map dictionary into a dataframe
genre = pd.DataFrame({'Genre_Name' : genre_map.keys(),
                     'Genre_Id': genre_map.values()})
genre.head()

,Genre_Name,Genre_Id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [25]:
# copying to another dataframe
title_basics = basics_df.copy()

In [ ]:
 # Extract certain columns
tmdb = combined[['imdb_id','budget','revenue', 'certification']]
tmdb

## **Discard unnecessary information**

In [12]:
# Drop following columns
title_basics = btitle_basics.drop(['originalTitle','isAdult', 'titleType','genres','genres_split'], axis = 1)
title_basics

,tconst,primaryTitle,startYear,endYear,runtimeMinutes
0,tt0000009,Miss Jerry,1894,NaN,45
1,tt0000574,The Story of the Kelly Gang,1906,NaN,70
2,tt0000591,The Prodigal Son,1907,NaN,90
3,tt0000679,The Fairylogue and Radio-Plays,1908,NaN,120
4,tt0001285,The Life of Moses,1909,NaN,50
...,...,...,...,...,...
151098,tt9914942,Life Without Sara Amat,2019,NaN,74
151099,tt9915872,The Last White Witch,2019,NaN,97
151100,tt9916170,The Rehearsal,2019,NaN,51
151101,tt9916190,Safeguard,2020,NaN,95


## **MySQL Database**

In [17]:
# loading mysql credentials
with open('/Users/julia/.secret/mysql.json') as f:
    login =json.load(f)
login.keys

<function dict.keys>

In [18]:
# Make connection string
connection_str= f"mysql+pymysql://{login['Login Name']}:{login['password']}@localhost/movies"
engine= create_engine(connection_str)

In [19]:
# create database
create_database(connection_str)

In [ ]:
# Transform to tables
title_basics.to_sql('title_basics', engine, if_exists = 'replace')
rating_df.to_sql('title_rating', engine, if_exists = 'replace')
title_genres.to_sql('title_genres', engine, if_exists = 'replace')
genre.to_sql('genres', engine, if_exists = 'replace')
tmdb.to_sql('tmdb_data', engine, if_exists = 'replace')

**Set primary key for title_basics tables**

In [ ]:
## Example
from sqlalchemy.types import *
## Calculate max string lengths for object columns
key_len = basics['tconst'].fillna('').map(len).max()
title_len = basics['primaryTitle'].fillna('').map(len).max()
## Create a schema dictonary using Sqlalchemy datatype objects
df_schema = {
    "tconst": String(key_len+1), 
    "primaryTitle": Text(title_len+1),
    'startYear':Float(),
    'endYear':Float(),
    'runtimeMinutes':Integer()}


In [ ]:
# Save to sql with dtype and index=False
title_basics.to_sql('title_basics',engine,dtype=df_schema,if_exists='replace',index=False)

In [ ]:
# Save to sql with dtype and index=False
title_basics.to_sql('title_basics',engine,dtype=df_schema,if_exists='replace',index=False)

In [ ]:
engine.execute('ALTER TABLE title_basics ADD PRIMARY KEY (`tconst`);')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM title_basics
LIMIT 5;"""
pd.read_sql(q, engine)

**Set primary key for ratings tables**

In [ ]:
# Calculate max string lengths for object columns
key_len = rating['tconst'].fillna('').map(len).max()

## Create a schema dictonary using Sqlalchemy datatype objects
df_2 = {
    "tconst": String(key_len+1),
    'averageRating':Float(),
    'numVotes':Integer()}

In [ ]:
# Save to sql with dtype and index=False
rating.to_sql('title_rating',engine,dtype=df_2,if_exists='replace',index=False)

In [ ]:
# update the table and set the primary key
engine.execute('ALTER TABLE title_rating ADD PRIMARY KEY (`tconst`);')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM title_rating
LIMIT 5;"""
pd.read_sql(q, engine)

**Set primary key for genre tables**

In [ ]:
# Set the dataframe index and use index=True 
genre.set_index('Genre_Id').to_sql('genres',engine,index=True, if_exists = 'replace')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM genres
LIMIT 5;"""
pd.read_sql(q, engine)

**Set primary key for tmdb tables**

In [ ]:
# Calculate max string lengths for object columns
key_len = tmdb['imdb_id'].fillna('').map(len).max()
cert_len = tmdb['certification'].fillna('').map(len).max()

## Create a schema dictonary using Sqlalchemy datatype objects
df_3 = {
    "imdb_id": String(key_len+1),
    'budget':Float(),
    'revenue':Float(),
    "certification":Text(cert_len+1)}

In [ ]:
# Save to sql with dtype and index=False
tmdb.to_sql('tmdb_data',engine,dtype=df_3,if_exists='replace',index=False)

In [ ]:
# update the table and set the primary key
engine.execute('ALTER TABLE tmdb_data ADD PRIMARY KEY (`imdb_id`);')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM tmdb_data
LIMIT 5;"""
pd.read_sql(q, engine)

In [ ]:
# Show all tables
q = """SHOW TABLES;"""
pd.read_sql(q, engine)